# In-context learning for Citation Prediction

In [1]:
import dspy
import pandas as pd
import ast
import numpy as np
import os
from numpy.linalg import norm
from tqdm import tqdm
from pathlib import Path
# from operator import add
from PyPDF2 import PdfReader
from openai import OpenAI

/Users/codyaldaz/repositories/dspy_scirepeval/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get the test data

In [2]:
query_candidate_data = pd.read_csv('darwin/test.qrel.cid', sep=' ', header=None, names=['query', 'candidate', 'bool'])

In [3]:

with open('darwin/qpaper_to_emb', 'r') as f:
    query_papers = [line.strip() for line in f]

with open('darwin/cpaper_to_emb', 'r') as f:
    candidate_papers = [line.strip() for line in f]

print(f'len(query_papers): {len(query_papers)}')
print(f'len(candidate_papers): {len(candidate_papers)}')


len(query_papers): 115
len(candidate_papers): 637


In [4]:
valid_rows = pd.DataFrame()
query_dir = 'darwin/query_papers'
candidate_dir = 'darwin/candidate_papers'
# Iterate over the rows of the data
for _, row in query_candidate_data.iterrows():
    query_file = os.path.join(query_dir, str(row['query']) + '.pdf')
    candidate_file = os.path.join(candidate_dir, str(row['candidate']) + '.pdf')

    # Check if both files exist
    if os.path.isfile(query_file) and os.path.isfile(candidate_file):
        # If both files exist, append the row to valid_rows
        valid_rows = valid_rows._append(row)

# Reset the index of valid_rows
valid_rows.reset_index(drop=True, inplace=True)
print(valid_rows.head())
print(f'Number of query candidate pairs with valid files: {len(valid_rows)}')

     query candidate  bool
0  3498240   1824499     1
1  3498240  53645322     0
2  3498240   1915951     0
3  3498240   3048298     0
4  3498240   3627503     0
Number of query candidate pairs with valid files: 651


In [5]:
data = [{"query_file": query_file, "candidate_file": candidate_file, "cites": bool(bool_)} for query_file, candidate_file, bool_ in zip(valid_rows['query'], valid_rows['candidate'], valid_rows['bool'])]
data = [dspy.Example(**x).with_inputs('query_file', 'candidate_file') for x in data]

def split_data(data, split_ratio, seed=42):
    np.random.seed(seed)
    indices = np.random.permutation(len(data))
    split_index = int(split_ratio * len(data))
    train_indices = indices[:split_index]
    test_indices = indices[split_index:]
    trainset = [data[i] for i in train_indices]
    testset = [data[i] for i in test_indices]
    return trainset, testset

trainset, testset = split_data(data, 0.2)


## Chunker

In [6]:
llm = dspy.OpenAI(model="gpt-3.5-turbo")
dspy.settings.configure(lm=llm, rm=None)

client = OpenAI(
    # this is also the default, it can be omitted
    api_key=os.environ['OPENAI_API_KEY'],
)

In [7]:
class Chunker:
    def __init__(self, context_window=3000, max_windows=5):
        self.context_window = context_window
        self.max_windows = max_windows
        self.window_overlap = 0.02

    def __call__(self, paper):
        snippet_idx = 0

        while snippet_idx < self.max_windows and paper:
            endpos = int(self.context_window * (1.0 + self.window_overlap))
            snippet, paper = paper[:endpos], paper[endpos:]

            next_newline_pos = snippet.rfind('\n')
            if paper and next_newline_pos != -1 and next_newline_pos >= self.context_window // 2:
                paper = snippet[next_newline_pos+1:] + paper
                snippet = snippet[:next_newline_pos]

            yield snippet_idx, snippet.strip()
            snippet_idx += 1

## DSPy Module

In [9]:
def get_embeddings(texts, model="text-embedding-3-small", save_file=None):
    if save_file and Path(save_file).exists():
        with open(save_file, 'r') as f:
            print(f"Loading embeddings from {save_file}")
            embeddings = [ast.literal_eval(line.strip()) for line in f]
        return embeddings
        
    try:
        response = client.embeddings.create(input=texts, model=model)
        embeddings = [embedding.embedding for embedding in response.data]
        if save_file: # Save the embeddings to a file
            with open(save_file, 'w') as f:
                print(f"Saving embeddings to {save_file}")
                for embedding in embeddings:
                    f.write(str(embedding) + '\n')
        return embeddings
    except Exception as e:
        print("Error during API call:", e)
        return []
    
def get_most_similar_chunk(query_embedding, candidate_embeddings, candidate_chunks):
    similarities = np.dot(candidate_embeddings, query_embedding) / (norm(candidate_embeddings, axis=1) * norm(query_embedding))
    most_similar_idx = np.argmax(similarities)
    return candidate_chunks[most_similar_idx]
    
    
class PredictCitation(dspy.Signature):
    __doc__ = """Predict if the two chunks are related by a citation."""
    query_chunk: str = dspy.InputField(desc='Query chunk to compare to the candidate chunk.')
    candidate_chunk: str = dspy.InputField(desc='Candidate chunk to compare to the query chunk.')
    answer: bool = dspy.OutputField(desc="either True or False", prefix="Answer:")


class PredictCitationAndResolve(dspy.Module):
    def __init__(self, context_window=3000, max_windows=5, resolve_function=any,
                 candidate_folder='darwin/candidate_papers', query_folder='darwin/query_papers'):
        super().__init__()
        
        self.chunk = Chunker(context_window=context_window, max_windows=max_windows)
        self.predict = dspy.TypedPredictor(PredictCitation)
        # self.predict = dspy.ChainOfThought(PredictCitation)
        self.resolve_function = resolve_function
        self.query_folder = query_folder
        self.candidate_folder = candidate_folder
        os.mkdir('embeddings', exist_ok=True)
        

    def forward(self, query_file, candidate_file):
        predictions = []
        
        # Get the text from the pdfs
        query_pdf = PdfReader(f'{self.query_folder}/{query_file}.pdf')
        query_text = ""
        for page in query_pdf.pages:
            page_text = page.extract_text()
            if page_text:
                query_text += page_text + " "  # Adding space to separate text between pages
        query_text = query_text.replace("\n", " ")
        
        candidate_pdf = PdfReader(f'{self.candidate_folder}/{candidate_file}.pdf')
        candidate_text = ""
        for page in candidate_pdf.pages:
            page_text = page.extract_text()
            if page_text:
                candidate_text += page_text + " "
        candidate_text = candidate_text.replace("\n", " ")
        
        # for each chunk in the paper
        query_chunks = [snippet for _, snippet in self.chunk(query_text)]
        candidate_chunks = [snippet for _, snippet in self.chunk(candidate_text)]
        
        # Create embeddings for the chunks
        candidate_embeddings = get_embeddings(candidate_chunks, save_file=f'embeddings/candidate_{candidate_file}.emb')
        query_embeddings = get_embeddings(query_chunks, save_file=f'embeddings/query_{query_file}.emb')
        
        for snippet, query_embedding in zip(query_chunks, query_embeddings):
            # Get the candidate chunk that is most similar to the snippet
            candidate_chunk = get_most_similar_chunk(query_embedding, candidate_embeddings, candidate_chunks)
            prediction = self.predict(query_chunk=snippet, candidate_chunk=candidate_chunk)
            # print(prediction)
            predictions.append(prediction.answer=='True')

        return dspy.Prediction(predictions=predictions, resolved=self.resolve_function(predictions))

## Tests

In [36]:
pipeline_chunking = PredictCitationAndResolve(max_windows=10)

In [20]:
# get an example
example = trainset[0]
example_x = example.inputs()
example_y = example.labels()
print(example_x)
print(example_y)

prediction = pipeline_chunking(**example_x)
print(prediction)
print(example_y.cites)

Example({'query_file': 1054114, 'candidate_file': '14525920'}) (input_keys=None)
Example({'cites': False}) (input_keys=None)
Loading embeddings from candidate_14525920.emb
Loading embeddings from query_1054114.emb
Prediction(
    predictions=[False, False, False, False, False],
    resolved=False
)
False


In [21]:
llm.inspect_history(n=5)





Predict if the two chunks are related by a citation.

---

Follow the following format.

Query Chunk: Query chunk to compare to the candidate chunk.
Candidate Chunk: Candidate chunk to compare to the query chunk.
Answer: either True or False (Respond with a single bool value)

---

Query Chunk: Pooled Motion Features for First-Person Videos M. S. Ryoo, Brandon Rothrock, and Larry Matthies Jet Propulsion Laboratory, California Institute of Technology, Pasadena, CA mryoo@jpl.nasa.gov Abstract In this paper, we present a new feature representation for ﬁrst-person videos. In ﬁrst-person video understanding (e.g., activity recognition), it is very important to capture both entire scene dynamics (i.e., egomotion) and salient lo- cal motion observed in videos. We describe a representa- tion framework based on time series pooling, which is de- signed to abstract short-term/long-term changes in feature descriptor elements. The idea is to keep track of how de- scriptor values are changing ov

In [33]:
from dspy.evaluate import Evaluate

def metric(example, result):
    '''Match metric'''
    # print('inside metric')
    # print(f'{example=}, {result=}')
    return 1 if example.cites == result.resolved else 0

In [35]:
evaluate = Evaluate(devset=trainset, metric=metric, num_threads=8, display_progress=True, display_table=0, max_errors=100)
evaluate(pipeline_chunking)

  0%|          | 0/130 [00:00<?, ?it/s]

Loading embeddings from candidate_13111019.emb
Loading embeddings from candidate_14525920.emb
Loading embeddings from query_12842965.emb
Loading embeddings from candidate_26230598.emb
Loading embeddings from candidate_17401731.emb
Loading embeddings from query_1054114.emb
inside metric
example=Example({'query_file': 12842965, 'candidate_file': '13111019', 'cites': True}) (input_keys={'candidate_file', 'query_file'}), result=Prediction(
    predictions=[False, False, False, False, False],
    resolved=False
)


Average Metric: 1 / 2  (50.0):   1%|          | 1/130 [00:03<07:09,  3.33s/it]

inside metric
example=Example({'query_file': 1054114, 'candidate_file': '14525920', 'cites': False}) (input_keys={'candidate_file', 'query_file'}), result=Prediction(
    predictions=[False, False, False, False, False],
    resolved=False
)
Loading embeddings from query_2197102.emb
Loading embeddings from query_21724528.emb


Average Metric: 2 / 3  (66.7):   2%|▏         | 3/130 [00:04<02:50,  1.34s/it]

...

Average Metric: 105.0 / 129  (81.4):  99%|█████████▉| 129/130 [02:13<00:03,  3.01s/it]

inside metric
example=Example({'query_file': 10148124, 'candidate_file': '8412218', 'cites': False}) (input_keys={'candidate_file', 'query_file'}), result=Prediction(
    predictions=[False, False, False, False, False, False, False, False, False, False],
    resolved=False
)


Average Metric: 105.0 / 130  (80.8): 100%|██████████| 130/130 [02:13<00:00,  1.03s/it]

Loading embeddings from candidate_2127100.emb
Loading embeddings from query_53079158.emb
inside metric
example=Example({'query_file': 53079158, 'candidate_file': '2127100', 'cites': True}) (input_keys={'candidate_file', 'query_file'}), result=Prediction(
    predictions=[False, False, False, False, False, False, False, False, False, False],
    resolved=False
)
Average Metric: 105.0 / 130  (80.8%)


80.77

In [24]:
llm.inspect_history(n=5)





Predict if the two chunks are related by a citation.

---

Follow the following format.

Query Chunk: Query chunk to compare to the candidate chunk.
Candidate Chunk: Candidate chunk to compare to the query chunk.
Answer: either True or False (Respond with a single bool value)

---

Query Chunk: erwithmoreneighborsinthefor- eignlanguage. Followingthemethodin Zhuetal.(2017),there will be no edge between Chinese entity 福斯特 (Foust)and English entity Pistons, which implies awrongfactthat 福斯特 (Foust)doesnotbelong toPistons. Ourmethodenrichesthemissingrela- tion between entities 福斯特 (Foust)and活塞队 (Pistons)in incomplete Chinese KB through cor- responding English common neighbors, Allstar, NBA,etc.,asillustratedinFigure 1. 4.2 Comparable Sentences Generation To supervise the cross-lingual representation learningofwords,weautomaticallygeneratecom- parable sentences as cross-lingual training data. Comparable sentences are not translated paired sentences,butsentenceswiththesametopicindif- fere

In [26]:
PdfReader('darwin/query_papers/53079158.pdf').pages[-1].extract_text()

'237TomasMikolov,QuocVLe,andIlyaSutskever.2013b.\nExploitingsimilaritiesamonglanguagesformachine\ntranslation. CoRR.\nTomas Mikolov, Ilya Sutskever, Kai Chen, Gregory S.\nCorrado, and Jeffrey Dean. 2013c. Distributed rep-\nresentations of words and phrases and their compo-\nsitionality. In NIPS.\nDavid Milne and Ian H. Witten. 2008. An effective,\nlow-cost measure of semantic relatedness obtained\nfromwikipedialinks. In AAAI.\nMike Mintz, Steven Bills, Rion Snow, and Daniel Ju-\nrafsky.2009. Distantsupervisionforrelationextrac-\ntionwithoutlabeleddata. In ACL/IJCNLP .\nAditya Mogadala and Achim Rettinger. 2016. Bilin-\ngual word embeddings from parallel and non-\nparallel corpora for cross-language text classiﬁca-\ntion. In HLT-NAACL .\nThien Huu Nguyen, Nicolas Fauceglia, Mariano Ro-\ndriguez Muro, Oktie Hassanzadeh, Alﬁo Massimil-\nianoGliozzo,andMohammadSadoghi.2016. Joint\nlearning of local and global features for entity link-\ningvianeuralnetworks. In COLING.\nSebastian Ruder, Iva